In [1]:
import sys
import os
import json
import argparse

import pickle
import tensorflow as tf

# Root directory of the project
ROOT_DIR = os.path.abspath("../")
FLWR_DIR = os.path.join(ROOT_DIR, 'src')
MASKRCNN_DIR = os.path.join(ROOT_DIR, 'maskrcnn')
WEIGHTS_DIR = os.path.join(ROOT_DIR, 'weights')

# Import Flower
sys.path.append(FLWR_DIR)
import flwr as fl
from flwr.common.logger import log

# Import Mask RCNN
sys.path.append(MASKRCNN_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import model as modellib, utils

# Path to trained weights file
COCO_WEIGHTS_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
WEIGHTS_SAVE_PATH= os.path.join(WEIGHTS_DIR, '00000.weights')

# Directory to save logs and model checkpoints, if not provided
# through the command line argument --logs
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")

In [23]:
class BalloonConfig(Config):
    NAME = "balloon"
    IMAGES_PER_GPU = 1
    NUM_CLASSES = 1 + 1  # Background + balloon
    # STEPS_PER_EPOCH = 100
    STEPS_PER_EPOCH = 10
    DETECTION_MIN_CONFIDENCE = 0.9

In [24]:
# Configuration for training 
config = BalloonConfig()

In [25]:
# Create model
model = modellib.MaskRCNN(mode="training", config=config, model_dir=DEFAULT_LOGS_DIR)
model.mode

'training'

In [26]:
exclude = [
    "mrcnn_class_logits",
    "mrcnn_bbox_fc",
    "mrcnn_bbox",
    "mrcnn_mask",
    "rpn_model"  # because anchor's ratio has been changed
]

# load coco weights
model.load_weights(
    COCO_WEIGHTS_PATH,
    by_name=True,
    exclude=exclude
)

In [8]:
import h5py
from tensorflow.python.keras.saving import hdf5_format

In [73]:
exclude = [
    "mrcnn_class_logits",
    "mrcnn_bbox_fc",
    "mrcnn_bbox",
    "mrcnn_mask",
    "rpn_model"  # because anchor's ratio has been changed
]

with h5py.File(COCO_WEIGHTS_PATH, mode='r') as f:
    layers = model.keras_model.layers
    
    # Exclude some layers
    layers = filter(lambda l: l.name not in exclude, layers)
    
    hdf5_format.load_weights_from_hdf5_group_by_name(f, layers)

# save them via pickel

In [75]:
weights = model.keras_model.get_weights()
len(weights)

385

In [77]:
try:
    print(f'save weights to {WEIGHTS_SAVE_PATH}')
    fs = open(WEIGHTS_SAVE_PATH, '+wb')
    pickle.dump(weights, fs )
    fs.close()
except:
    print(f'Error when trying to saving weights to {WEIGHTS_SAVE_PATH}')

save weights to /home/ps101/projekte/flower_client/autovikki/weights/00000.weights


# Try to load weights

In [80]:
weights = []
try:
    fs = open(WEIGHTS_SAVE_PATH, 'rb')
    weights = pickle.load(fs)
    fs.close()
except:
    print('ERROR')

len(weights)

385

In [81]:
model.keras_model.set_weights(weights)

## Download balloon dataset

In [91]:
import requests

URL = 'https://raw.githubusercontent.com/earnestt1234/seedir/master/setup.py'

In [92]:
r = requests.get(URL)

In [93]:
r


<Response [200]>

In [86]:
r = requests.get('https://raw.github.com/kennethreitz/requests/master/README.rst')

In [87]:
r

<Response [404]>